お疲れ様です。細かい背景などについては後に説明するとしてなんとなくイメージがつく感じで内容を書きたいと思います。
治療を行った群（治療群)$X_{treat}$,行わなかった群(対照群)$X_{ctrl}$の間の予後(死亡率)Yを考えます。単純に考えると
$Y=X_{treat}-X_{ctrl}$ですが、これができるのは治療群、対照群をランダムに割つけた場合のみとなります。なぜなら、治療を行っている患者の方が状態がもともと悪いから$X_{treat}-X_{ctrl}=0$であっても、治療した結果0になったと捉える事も可能であるからです。

この為両群の間で同程度の重症度の患者通しを比較して治療効果を見積もることになります。手法としては層別解析や傾向スコアマッチングといったものが使われます。
詳細は
https://www.iwanami.co.jp/book/b243764.html
や
https://www.amazon.co.jp/dp/4000069721/ref=asc_df_40000697212586329/?tag=jpgo-22&creative=9303&creativeASIN=4000069721&linkCode=df0&hvadid=295699891199&hvpos=1o1&hvnetw=g&hvrand=15820861973558530482&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=1009280&hvtargid=pla-525812123008&th=1&psc=15820861973558530482
にありますが、長くなるのでここでは省きます。
この方法はかなり確立した方法ですが問題点として、これらの手法は「薬が効果あるかどうか？」という疑問に対し、「治療群と対照群」どちらにも属し得る患者に対しての治療効果しか調べていないにも関わらず、この結果をもって薬が効果ある、効果ないの二元論で議論が行われている状況です。

長くなるので、少し話をとばしますが
$Y = \beta_{eff} X_{treat}+ \Sigma_{i}\beta_{i}X_{i}+N(0,\sigma_{1})$
のような回帰式を考えます。
 $\beta_{eff}$を効果量と呼びます。<br>
 $x_{treat}$ はダミー変数で治療群なら1みたいなもの、
 $\Sigma_{i}\beta_{i}X_{i}$は年齢、性別、基礎疾患などの特徴量
 最後の項は誤差で、上記の問題は $\beta_{eff}$の推定問題となります。
 

ここで、治療群に入るかどうかについて以下の式を導入します。<br>
$X_{treat} = N(\Sigma_{j}X_{j}\beta^{'}_{j}+b, \sigma_{2})$
<br> ようするに$X_{treat}$自体も他の変数に依存している。

一番単純なモデルとして<br>
$y = effect * x1 + \beta_{2} * x2 + N(0,0.2)$<br>
$x2 = N(x1*1.2+0.5, 0.5)$
を作ってみます。

In [3]:
import numpy as np
import random
from numpy.random import *
import matplotlib.pyplot as plt 
#np.random.seed(100)
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import tensorflow as tf
import csv

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

rng = np.random.RandomState(1234)

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
x1 = np.random.binomial(1,0.5,size=1000)
x2 = np.random.normal(x1*0.4+0.5,0.2)
effect = 1.5
beta2 = 0.8
eps = 1e-10
b = np.random.normal(0,0.2)


def kaiki(x1=x1,x2=x2,effect=effect,beta2=beta2,b=b):
    return effect*x1+beta2*x2+b
y = kaiki(x1,x2)
data = (x1,x2,y)

dataでは、治療の有無の情報がx1に。リスク因子がx2。その患者の予後がyとして入っています。
ここからx1の回帰係数を求める問題を考えます。（もちろんこれは単純な設定なので普通にlmしたら求まるのですが）
これを生成モデルで解いて、その後非常に複雑な式の場合にもちゃんと解ける事ができるようにするのが目標です。

生成モデルを使うメリットは、一旦モデルを学習してしまえば
1.治療群、対照群の患者をbackgroundが揃った状態で生成し効果量を単純な引き算で計算できる
2.潜在変数を動かすことによりもっとも治療が効果的な患者群、有効でない患者群を探す事ができる
3.未知の患者についてもbackground情報を入れたら効果量を算定できる
という事です。

モデルとしてはタグありの半教師あり学習で。昨日の講義で出て来たMNISTのモデルが基本です。また、類似研究としては薬の開発でもっとも効果のある薬を潜在空間内で探るというものが過去にありました。後ほど添付します。

Roadmapとしては<br>
1.まず上記もっとも単純なモデルで生成モデルを作成し、効果量の推定がきちんと行える事を確認する。<br>
2.モデルに交互作用項をつけたり、表に出てこない説明変数を加えたりしてより複雑なモデルを作成する。<br>
--ここまでが最低ライン<br>
3.傾向スコアマッチングと比較してその差を比較する<br>
4.ベイズ最適化によりより洗練されたモデル作成およびもっとも効果のある患者群の探索を行う<br>
5.Bayesian DNNもしくはガウス過程により治療効果の期待値だけでなく分散まで表現できるようにする<br>
--オプション<br>
6.過去に行われた臨床データを使い、傾向スコアを使った解析と比較し優劣を検討<br>
7.学習後のモデルをアプリにし使用感を検討<br>